# 🚀 İnteraktif JetX Model Eğitmeni (v5.0 - Colab Formları ile Kararlı Sürüm)

### 🛠️ 1. Kurulum (Tek Adım)

**Lütfen sadece bu hücreyi çalıştırın ve tamamlanmasını bekleyin.**

Bu hücre, gerekli tüm adımları otomatik olarak gerçekleştirir:

1.  **Projeyi Klonlar/Günceller:** Gerekli kodları GitHub'dan çeker.
2.  **Kütüphaneleri Yükler:** `requirements_enhanced.txt` dosyasındaki tüm bağımlılıkları kurar.
3.  **Python Path'i Ayarlar:** Proje kaynak kodunun import edilebilmesini sağlar.

In [ ]:
import os
import sys
import subprocess

def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "stash"], check=True, capture_output=True)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
        subprocess.run(["git", "stash", "pop"], check=False, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

try:
    setup_environment()
    print("\n🎉 Kurulum başarıyla tamamlandı! Sonraki hücreye geçebilirsiniz.")
except Exception as e:
    print(f"\n❌ Kurulum sırasında bir hata oluştu: {e}")

## 📊 2. MLflow Deney Takibini Başlatma (İsteğe Bağlı)

Eğitim sürecini ve metrikleri detaylı olarak izlemek isterseniz bu hücreyi çalıştırabilirsiniz. Sizden bir `ngrok` token isteyecektir.

In [ ]:
from getpass import getpass
from colab.health_check import ColabHealthCheck
import time
from pyngrok import ngrok

NGROK_AUTH_TOKEN = getpass('Lütfen ngrok authtokeninizi girin: ')

try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok token ayarlandı.")
    get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')
    time.sleep(5)
    public_url = ngrok.connect(5000, "http").public_url
    print("\n✅ MLflow arayüzü için public URL oluşturuldu.")
    print(f"👇 Deneyleri izlemek için tıklayın: {public_url}")
except Exception as e:
    print(f"\n❌ MLflow başlatılamadı. Lütfen authtoken'ınızı kontrol edin. Hata: {e}")

In [ ]:
#@title 🚀 3. Modelleri Seç, Eğit ve İndir
#@markdown ---
#@markdown ### **Eğitmek istediğiniz modelleri seçin**
#@markdown Lütfen aşağıdaki listeden eğitmek istediğiniz modelleri işaretleyin ve ardından hücreyi çalıştırın (▶️ butonu).
#@markdown ---

# --- Gerekli Kütüphaneler ve Ayarlar ---
import torch
from IPython.display import display, FileLink
import traceback
from src.training.pipeline_manager import PipelineManager
from src.config.settings import get_all_models

# --- Mevcut Modelleri Yükle ve Formu Oluştur ---
try:
    all_models_list = get_all_models()
    print(f"✅ MEVCUT MODELLER: {', '.join(all_models_list)}")
except Exception as e:
    print(f"❌ Modeller yüklenemedi. `config.yaml` dosyasını kontrol edin. Hata: {e}")
    all_models_list = []

#@markdown
#@markdown **Model Seçimi:**
N_Beats = False #@param {type:"boolean"}
TFT = False #@param {type:"boolean"}
LSTM = False #@param {type:"boolean"}

# --- Ana Eğitim Fonksiyonu ---
def run_training_from_form(selected_models):
    if not selected_models:
        print("\n⚠️ Lütfen yukarıdan en az bir model seçip hücreyi tekrar çalıştırın.")
        return

    print(f"\n🚀 Seçilen modellerle eğitim başlatılıyor: {', '.join(selected_models)}")
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Cihaz: {device.upper()}")

    try:
        pipeline_manager = PipelineManager(models_to_train=selected_models, device=device)
        trained_model_paths = pipeline_manager.run_pipeline()

        print("\n" + "="*50 + "\n🎉 EĞİTİM TAMAMLANDI 🎉\n" + "="*50)

        if trained_model_paths:
            print("\n⬇️ EĞİTİLMİŞ MODELLERİ İNDİR ⬇️")
            for path in trained_model_paths:
                display(FileLink(path))
        else:
            print("\n❌ Eğitilmiş model dosyası bulunamadı. İndirme linki oluşturulamadı.")

    except Exception as e:
        print(f"\n❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
        traceback.print_exc()

# --- Formdan Gelen Değerlerle Eğitimi Başlat ---
if all_models_list:
    # Formdan seçilen modelleri topla
    models_to_train = []
    if 'N-Beats' in all_models_list and N_Beats:
        models_to_train.append('N-Beats')
    if 'TFT' in all_models_list and TFT:
        models_to_train.append('TFT')
    if 'LSTM' in all_models_list and LSTM:
        models_to_train.append('LSTM')
    
    run_training_from_form(models_to_train)